---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [2]:
import pandas as pd
import numpy as np
import re

def answer_one():
    energy = get_energy()
    GDP = get_GDP()
    ScimEn = get_ScimEn()
    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
           'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
           '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    res=pd.merge(energy, ScimEn)
    res=res.query("Rank <= 15")
    res=pd.merge(res, GDP, left_on="Country", right_on="Country Name")
    res.index = res["Country"].values
    return res.loc[:,columns ]
    
def get_energy():
    dicts = {"Republic of Korea": "South Korea",
             "United States of America": "United States",
             "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
             "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy = pd.read_excel("Energy Indicators.xls", sheetname="Energy",skiprows=17, skip_footer=38).iloc[:, 2:]
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy["Energy Supply"]=energy["Energy Supply"].replace({"...":np.NaN})
    energy["Energy Supply per Capita"]=energy["Energy Supply per Capita"].replace({"...":np.NaN})
    energy["Energy Supply"] = energy["Energy Supply"]*1000000
    for i in range(len(energy)):
        country =  energy["Country"][i].strip()
        temp = re.sub(r"\d", "",country)
        temp =  re.sub(r"\(.*\)", "" , temp)
        temp = temp.strip()
        energy["Country"] = energy["Country"].replace(country, temp)
    energy["Country"] = energy["Country"].replace(dicts)
    return energy

def get_GDP():
    GDP = pd.read_csv("world_bank.csv", skiprows=4)
    dicts = {"Korea, Rep.": "South Korea", 
         "Iran, Islamic Rep.": "Iran",
         "Hong Kong SAR, China": "Hong Kong"}
    GDP["Country Name"] = GDP["Country Name"].replace(dicts)
    return GDP

def get_ScimEn():
    ScimEn = pd.read_excel("scimagojr-3.xlsx", sheetname="Sheet1")
    return ScimEn

data  = answer_one()

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [38]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [49]:
def answer_two():
    energy = get_energy()
    GDP = get_GDP()
    ScimEn = get_ScimEn()
    res=pd.merge(energy, ScimEn)
    res=pd.merge(res, GDP, left_on="Country", right_on="Country Name")
    a=set(energy["Country"].values)
    b=set(GDP["Country Name"].values)
    c=set(ScimEn["Country"].values)
    return len( a.union(b).union(c)) - len(res)

answer_two()

156

<br>

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [40]:
def answer_three():
    Top15 = answer_one()
    res=Top15.iloc[:, 10:]
    res=res.mean(axis=1)
    res.name = "avgGDP"
    res=res.sort_values(ascending=False)
    return res
answer_three()

United States         1.562297e+13
China                 6.940046e+12
Japan                 5.176323e+12
Germany               3.532694e+12
United Kingdom        2.724782e+12
France                2.688773e+12
Italy                 2.130288e+12
Brazil                1.989028e+12
Russian Federation    1.656010e+12
Canada                1.611540e+12
India                 1.594349e+12
Spain                 1.406409e+12
Australia             1.202763e+12
South Korea           1.165810e+12
Iran                  4.369708e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [41]:
def answer_four():
    Top15 = answer_one()
    country = answer_three().index[5]
    start = Top15.loc[country, "2006"]
    end = Top15.loc[country, "2015"]
    return end-start 

answer_four()

108550097312.71973

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [6]:
def answer_five():
    Top15 = answer_one()
    return float(Top15.mean()["Energy Supply per Capita"])

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [7]:
def answer_six():
    Top15 = answer_one()
    max_value = Top15.max()["% Renewable"]
    country = Top15[Top15["% Renewable"]==max_value].index[0]
    return country, max_value
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [8]:
def answer_seven():
    Top15 = answer_one()
    ratio = Top15["Self-citations"]/Top15["Citations"]
    Top15["ratio"] = ratio
    max_value = Top15.max()["ratio"]
    country = Top15[Top15["ratio"]==max_value].index[0]
    return country, max_value

answer_seven()

('China', 0.68143547080669364)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [28]:
def answer_eight():
    Top15 = answer_one()
    population = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
    Top15["population"] = population
    Top15=Top15.sort_values(by="population", ascending=False)
    return Top15.index[2]

answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [38]:
def answer_nine():
    Top15 = answer_one()
    population = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
    Top15["population"] = population
    Top15["citable per capital"] = Top15["Citable documents"]/Top15["population"]
    a=Top15["citable per capital"]
    b=Top15["population"]
    return a.corr(b)

answer_nine()

-0.53724989764594078

In [51]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x=Top15['Energy Supply per Capita'],y=Top15['Citable docs per Capita'] , kind='scatter', xlim=[0, 0.0006])


In [50]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [49]:
def answer_ten():
    Top15 = answer_one()
    median= Top15.median()["% Renewable"]
    arr = []
    for i in Top15["% Renewable"]:
        if(i >=median):
            arr.append(1)
        else:
            arr.append(0)
    hr = pd.Series(arr, Top15.index, name="HighRenew")
    return hr


answer_ten()

Australia             0
Brazil                1
Canada                1
China                 1
France                1
Germany               1
India                 0
Iran                  0
Italy                 1
Japan                 0
South Korea           0
Russian Federation    1
Spain                 1
United Kingdom        0
United States         0
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [63]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15 = answer_one()
    population = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
    a=population.groupby(ContinentDict)
    p_sum = a.sum()
    p_size = a.size()
    p_mean = a.mean()
    p_std = a.std()
    con = p_size.index
    return pd.DataFrame({"sum":p_sum, "size":p_size, "mean":p_mean, "std":p_std})


answer_eleven()

,mean,size,std,sum
Asia,5.797333e+08,5,6.790979e+08,2.898666e+09
Australia,2.331602e+07,1,NaN,2.331602e+07
Europe,7.632161e+07,6,3.464767e+07,4.579297e+08
North America,1.764276e+08,2,1.996696e+08,3.528552e+08
South America,2.059153e+08,1,NaN,2.059153e+08


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [56]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    vals = Top15["% Renewable"]
    Top15["bin"] = pd.cut(vals, 5)
    #return Top15.groupby([ContinentDict, "bin"])["bin"]

Top15 = answer_one()
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
vals = Top15["% Renewable"]
Top15["bin"] = pd.cut(vals, 5)
Top15[["% Renewable","bin"]]

Australia           % Renewable             11.8108
                    bin             (2.212, 15.753]
Brazil              % Renewable              69.648
                    bin            (56.174, 69.648]
Canada              % Renewable             61.9454
                    bin            (56.174, 69.648]
China               % Renewable             19.7549
                    bin            (15.753, 29.227]
France              % Renewable             17.0203
                    bin            (15.753, 29.227]
Germany             % Renewable             17.9015
                    bin            (15.753, 29.227]
India               % Renewable             14.9691
                    bin             (2.212, 15.753]
Iran                % Renewable             5.70772
                    bin             (2.212, 15.753]
Italy               % Renewable             33.6672
                    bin            (29.227, 42.701]
Japan               % Renewable             10.2328
            

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [65]:
def answer_thirteen():
    Top15 = answer_one()
    population = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
    Top15["PopEst"] = population
    arr = []
    for country in Top15.index:
        arr.append("{:,}".format(Top15["PopEst"][country]))
    return pd.Series(arr, Top15.index, name="PopEst")

answer_thirteen()

Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
France                  63,837,349.39759036
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
South Korea            49,805,429.864253394
Russian Federation            143,500,000.0
Spain                    46,443,396.2264151
United Kingdom         63,870,967.741935484
United States          317,615,384.61538464
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [53]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [54]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!